In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from collections import defaultdict
import pickle
import json
import csv
from ast import literal_eval
from scipy.spatial import distance

# Read Data 

In [34]:
with open('pickleData/pd_business.pkl', 'rb') as f:
    pd_business = pickle.load(f)

In [35]:
pd_business.head()

,Restaurant_categories,attributes_list,location,rate_vec
business_id,,,,
QXAEGFB4oINsVuTFxEYKFQ,"[specialty food, dim sum, imported food, food,...","[RestaurantsReservations, GoodForMeallunch, Go...","[Mississauga, ON]","[2.5, 128]"
gnKjwL_1w79qoiV3IC_xQQ,"[sushi bars, japanese]","[GoodForKids, NoiseLevelaverage, GoodForMeallu...","[Charlotte, NC]","[4.0, 170]"
1Dfx3zM-rW4n-31KeC8sJg,"[breakfast & brunch, mexican, tacos, tex-mex, ...","[RestaurantsTakeOut, WiFino, RestaurantsAttire...","[Phoenix, AZ]","[3.0, 18]"
fweCYi8FmbJXHCqLnwuk8w,"[italian, pizza, chicken wings]","[RestaurantsPriceRange22, BusinessAcceptsCredi...","[Mentor-on-the-Lake, OH]","[4.0, 16]"
tstimHoMcYbkSC4eBA1wEg,"[mexican, patisserie/cake shop, food, bars, ni...","[BikeParking, BusinessParkinglot, BusinessAcce...","[Las Vegas, NV]","[4.5, 184]"


In [36]:
print(pd_business.loc['tstimHoMcYbkSC4eBA1wEg'])

Restaurant_categories    [mexican, patisserie/cake shop, food, bars, ni...
attributes_list          [BikeParking, BusinessParkinglot, BusinessAcce...
location                                                   [Las Vegas, NV]
rate_vec                                                        [4.5, 184]
Name: tstimHoMcYbkSC4eBA1wEg, dtype: object


In [7]:
print(pd_business.iloc[0].name)
print(pd_business.iloc[0,0])
print(pd_business.iloc[0,1])
print(pd_business.iloc[0,2])
print(pd_business.iloc[0,3])

QXAEGFB4oINsVuTFxEYKFQ
['specialty food', 'dim sum', 'imported food', 'food', 'chinese', 'ethnic food', 'seafood']
['RestaurantsReservations', 'GoodForMeallunch', 'GoodForMealdinner', 'BusinessParkinglot', 'Caters', 'NoiseLevelloud', 'RestaurantsTableService', 'RestaurantsTakeOut', 'RestaurantsPriceRange22', 'Ambiencecasual', 'WiFino', 'GoodForKids', 'Alcoholfull_bar', 'RestaurantsAttirecasual', 'RestaurantsGoodForGroups']
['Mississauga', 'ON']
[2.5, 128]


In [8]:
print("# of restaurants: {}".format(pd_business.shape[0]))

# of restaurants: 44100


In [17]:
pd_review = pd.read_csv('review.csv')
pd_review.head()

,user_id,business_id,stars
0,c6HT44PKCaXqzN_BdgKPCw,u8C8pRvaHXg3PgDrsUHJHQ,5.0
1,nkN_do3fJ9xekchVC-v68A,u8C8pRvaHXg3PgDrsUHJHQ,4.0
2,nkN_do3fJ9xekchVC-v68A,EZOoB2D8uQHV_gJoGCMTxQ,4.0
3,62GNFh5FySkA3MbrQmnqvg,ydUqgWsF3F27TbauOyib0w,4.0
4,Ud72j_rglYmXrL_O8zCdoA,LkMtMHVetws5_7QfRjPtlg,4.0


In [18]:
# pd_review.set_index(['user_id', 'business_id'], inplace=True)
# pd_review.head()

In [19]:
# print(pd_review.loc[('62GNFh5FySkA3MbrQmnqvg','ydUqgWsF3F27TbauOyib0w'),'stars'].iloc[0])
# print(('g','a') in pd_review.index)

In [20]:
with open('pickleData/userToRes_dict.pkl', 'rb') as f:
    userToRes_dict = pickle.load(f)

In [21]:
with open('pickleData/resToUser_dict.pkl', 'rb') as f:
    resToUser_dict = pickle.load(f)

In [22]:
print("# of users: {}".format(len(userToRes_dict)))
print("# of restuarants: {}".format(len(resToUser_dict)))

# of users: 1112564
# of restuarants: 44100


In [23]:
userNumRates = defaultdict(int)
extreme_rate_users = set()
for u in userToRes_dict:
    userNumRates[len(userToRes_dict[u])] += 1
    if len(userToRes_dict[u]) <= 3 or len(userToRes_dict[u]) >= 101:
        extreme_rate_users.add(u)
print(sorted(userNumRates.items(), key=lambda tp:tp[0]))
print(len(extreme_rate_users))
print(list(extreme_rate_users)[:5])

[(1, 610807), (2, 187346), (3, 91244), (4, 53610), (5, 34660), (6, 24079), (7, 17826), (8, 13401), (9, 10325), (10, 8358), (11, 6826), (12, 5545), (13, 4746), (14, 4026), (15, 3407), (16, 2908), (17, 2526), (18, 2177), (19, 1941), (20, 1791), (21, 1606), (22, 1346), (23, 1284), (24, 1085), (25, 1057), (26, 983), (27, 843), (28, 801), (29, 776), (30, 714), (31, 644), (32, 641), (33, 551), (34, 479), (35, 499), (36, 470), (37, 447), (38, 402), (39, 345), (40, 360), (41, 331), (42, 325), (43, 282), (44, 281), (45, 286), (46, 279), (47, 253), (48, 212), (49, 240), (50, 232), (51, 203), (52, 189), (53, 222), (54, 190), (55, 192), (56, 158), (57, 156), (58, 129), (59, 161), (60, 137), (61, 133), (62, 139), (63, 135), (64, 135), (65, 129), (66, 142), (67, 103), (68, 103), (69, 104), (70, 109), (71, 93), (72, 95), (73, 91), (74, 92), (75, 80), (76, 86), (77, 88), (78, 77), (79, 86), (80, 77), (81, 70), (82, 70), (83, 63), (84, 72), (85, 68), (86, 53), (87, 51), (88, 71), (89, 58), (90, 58), (9

In [24]:
print('before drop extreme pd_review # of rows: {}'.format(pd_review.shape[0]))
extreme_index = list()
def find_extreme(row):
    if row['user_id'] in extreme_rate_users:
        extreme_index.append(row.name)
pd_review.apply(find_extreme, axis=1)
pd_review.drop(extreme_index, axis=0, inplace=True)
pd_review.head()

before drop extreme pd_review # of rows: 3954330


,user_id,business_id,stars
1,nkN_do3fJ9xekchVC-v68A,u8C8pRvaHXg3PgDrsUHJHQ,4.0
2,nkN_do3fJ9xekchVC-v68A,EZOoB2D8uQHV_gJoGCMTxQ,4.0
11,HLaSqQMDVvlcFPGJL_kGCA,VMPSdoBgJuyS9t_x_caTig,5.0
12,HLaSqQMDVvlcFPGJL_kGCA,GjkgXXicSJokejRge-g3xA,5.0
13,HLaSqQMDVvlcFPGJL_kGCA,IXXERocY1bqGwRllcy8J2w,5.0


In [25]:
print('after drop extreme pd_review # of rows: {}'.format(pd_review.shape[0]))

after drop extreme pd_review # of rows: 2348407


In [26]:
normalRateRes = set(pd_review['business_id'])
print("# of restuarants after filter extreme rate users: {}".format(len(normalRateRes)))

# of restuarants after filter extreme rate users: 43756


In [27]:
noRateRes = list()
def findNoRateRes(row):
    if row.name not in normalRateRes:
        noRateRes.append(row.name)
pd_business.apply(findNoRateRes, axis=1)
pd_business.drop(noRateRes, axis=0, inplace=True)
print("# of restaurants after filter out no rate restuarant: {}".format(pd_business.shape[0]))

# of restaurants after filter out no rate restuarant: 43756


In [28]:
userToRes_dict = dict(userToRes_dict)
resToUser_dict = dict(resToUser_dict)

for u in extreme_rate_users:
    userToRes_dict.pop(u)

for r in noRateRes:
    resToUser_dict.pop(r)
        
print("# of users: {}".format(len(userToRes_dict)))
print("# of restuarants: {}".format(len(resToUser_dict)))

# of users: 221182
# of restuarants: 43756


In [29]:
pd_review.to_pickle("pd_review.pkl")

In [30]:
pd_business.to_pickle("pd_business.pkl")

In [31]:
with open('userToRes_dict.pickle', 'wb') as f:
    pickle.dump(userToRes_dict, f, pickle.HIGHEST_PROTOCOL)

In [32]:
with open('resToUser_dict.pickle', 'wb') as f:
    pickle.dump(resToUser_dict, f, pickle.HIGHEST_PROTOCOL)

# Define Distance Functions

In [19]:
def cal_cos_sim(vec1, vec2):
    assert isinstance(vec1, list)
    assert isinstance(vec2, list)
    dis = distance.cosine(vec1, vec2)
    return 1 - dis

print(cal_cos_sim([1, 0, 0], [0, 1, 0]))
print(cal_cos_sim([3, 45, 7, 2], [2, 54, 13, 15]))

0.0
0.97228425171235


In [20]:
def Jaccard(vec1, vec2):
    assert isinstance(vec1, list)
    assert isinstance(vec2, list)
    set1 = set(vec1)
    set2 = set(vec2)
    inter = len(set1.intersection(set2))
    uni = len(set1.union(set2))
    return inter / uni

print(Jaccard(['pizza', 'hot'], ['pizza', 'orange', 'apple']))

0.25


In [21]:
%%time
print(Jaccard(pd_business.iloc[0,0],pd_business.iloc[1,0]) + Jaccard(pd_business.iloc[0,1],pd_business.iloc[1,1]) + Jaccard(pd_business.iloc[0,2],pd_business.iloc[1,2]) + cal_cos_sim(pd_business.iloc[0,3],pd_business.iloc[1,3]))

1.5238015385975454
Wall time: 2.99 ms


# Content Base Recommendation Model

In [22]:
class ContentRecomm:
    def __init__(self, userToRes_dict, pd_business, pd_review, ratio=[0.3, 0.1, 0.4, 0.2]):
        self.userToRes_dict = userToRes_dict
        self.pd_business = pd_business
        self.pd_review = pd_review
        self.ratio = ratio
        
        self.all_sim_dict = defaultdict(list)
    
    @staticmethod
    def cal_cos_sim(vec1, vec2):
        assert isinstance(vec1, list)
        assert isinstance(vec2, list)
        dis = distance.cosine(vec1, vec2)
        return 1 - dis
    
    @staticmethod
    def Jaccard(vec1, vec2):
        assert isinstance(vec1, list)
        assert isinstance(vec2, list)
        set1 = set(vec1)
        set2 = set(vec2)
        inter = len(set1.intersection(set2))
        uni = len(set1.union(set2))
        return inter / uni
    
    def makeRecomm(self, resId, top_N):
        if resId not in self.all_sim_dict:
            for index, row in self.pd_business.iterrows():
                if resId != index:
                    sim_cat = self.ratio[0] * ContentRecomm.Jaccard(self.pd_business.loc[resId,'Restaurant_categories'],self.pd_business.loc[index,'Restaurant_categories'])
                    sim_attr = self.ratio[1] * ContentRecomm.Jaccard(self.pd_business.loc[resId,'attributes_list'],self.pd_business.loc[index,'attributes_list'])
                    sim_loc = self.ratio[2] * ContentRecomm.Jaccard(self.pd_business.loc[resId,'location'],self.pd_business.loc[index,'location'])
                    sim_rate = self.ratio[3] * ContentRecomm.cal_cos_sim(self.pd_business.loc[resId,'rate_vec'],self.pd_business.loc[index,'rate_vec'])
                    self.all_sim_dict[resId].append((index,sim_cat+sim_attr+sim_loc+sim_rate))

            self.all_sim_dict[resId].sort(key=lambda tp:tp[1], reverse=True)
        
        return [tp[0] for tp in self.all_sim_dict[resId][:top_N]]
    
#     def cal_recall_score(self,userId, resId, top_k, top_N, threshold=2):
#         top_N_recomm = self.makeRecomm(resId, top_N)
#         relevant = set()
#         top_N_recomm_rate = list()
#         for index, mov in enumerate(top_N_recomm):
#             if (userId, mov) in pd_review.index:
#                 if self.pd_review.loc[(userId, mov),'stars'].iloc[0] > threshold:
#                     relevant.add(mov)
#             top_N_recomm_rate.append((mov, pred_rate(userId, mov)))
#         top_N_recomm_rate.sort(key=lambda tp: tp[1], reverse=True)
#         rec_at_top_k = set([tp[0] for tp in top_N_recomm_rate[:top_k]])
#         return len(relevant.intersection(rec_at_top_K)) / len(relevant)
    
    def simpleRecall(self, userId, resId, top_N):
        '''
        # restaurants that user has been that are also in top_N recommendations / top_N 
        '''
        top_N_recomm = set(self.makeRecomm(resId, top_N))
        return len(top_N_recomm.intersection(set(self.userToRes_dict[userId]))) / top_N
        

In [23]:
import random
randomTest = random.randrange(pd_review.shape[0])
test_user = pd_review.iloc[randomTest, 0]
test_res = pd_review.iloc[randomTest, 1]
top_N = 5
contentModel = ContentRecomm(userToRes_dict, pd_business, pd_review)
top_N_recomm = contentModel.makeRecomm(test_res, top_N)
print(top_N_recomm)

['MUjlAtBNJR--7ai5pITbMg', '6LR0Eg5EAMEIES7CY7ITOg', 'ERnG-1q3igX3VSgm5uLZ6A', 'p1ohbPwI1QfjHQ2zRup5OA', 'qUWqjjjfpB2-4P3He5rsKw']


In [53]:
test_user = pd_review.iloc[randomTest, 0]
test_res = pd_review.iloc[randomTest, 1]
for r in userToRes_dict[test_user][0:4]:
    print('Restaurant id: {}'.format(pd_business.loc[r].name))
    print('Food categories: {}'.format(pd_business.loc[r, 'Restaurant_categories']))
    print('Attributes: {}'.format(pd_business.loc[r, 'attributes_list']))
    print('Location: {}'.format(pd_business.loc[r, 'location']))
    print('Rate, # of rates: {}'.format(pd_business.loc[r, 'rate_vec']))
    print("=====================================================================")
for r in userToRes_dict[test_user][6:8]:
    print('Restaurant id: {}'.format(pd_business.loc[r].name))
    print('Food categories: {}'.format(pd_business.loc[r, 'Restaurant_categories']))
    print('Attributes: {}'.format(pd_business.loc[r, 'attributes_list']))
    print('Location: {}'.format(pd_business.loc[r, 'location']))
    print('Rate, # of rates: {}'.format(pd_business.loc[r, 'rate_vec']))
    print("=====================================================================")

Restaurant id: oqJe5o_4J0xkhvTSTP9UNQ
Food categories: ['sushi bars', 'japanese']
Attributes: ['RestaurantsTakeOut', 'RestaurantsGoodForGroups', 'BusinessParkingstreet', 'RestaurantsReservations', 'RestaurantsPriceRange22', 'NoiseLevelaverage', 'Alcoholfull_bar', 'GoodForKids', 'GoodForMeallunch', 'GoodForMealdinner', 'RestaurantsDelivery', 'Caters', 'BikeParking', 'RestaurantsAttirecasual', 'WiFifree']
Location: ['Toronto', 'ON']
Rate, # of rates: [3.5, 112]
Restaurant id: td9FZybutwNG7DgocHCiXA
Food categories: ['dim sum', 'chinese']
Attributes: ['NoiseLevelquiet', 'RestaurantsPriceRange23', 'RestaurantsGoodForGroups', 'GoodForKids', 'RestaurantsReservations', 'Alcoholfull_bar', 'GoodForMeallunch', 'GoodForMealdinner', 'GoodForMealbrunch', 'Ambienceclassy', 'Ambienceupscale', "RestaurantsAttire'casual'", 'Caters', 'RestaurantsTakeOut', "WiFi'free'", 'BusinessAcceptsCreditCards']
Location: ['Toronto', 'ON']
Rate, # of rates: [3.5, 191]
Restaurant id: 5EMadlOW4sta6CEtJHf9uQ
Food catego

In [24]:
randList = [random.randrange(pd_review.shape[0]) for i in range(10)]

In [25]:
print('Restaurant id: {}'.format(pd_business.loc[test_res].name))
print('Food categories: {}'.format(pd_business.loc[test_res, 'Restaurant_categories']))
print('Attributes: {}'.format(pd_business.loc[test_res, 'attributes_list']))
print('Location: {}'.format(pd_business.loc[test_res, 'location']))
print('Rate, # of rates: {}'.format(pd_business.loc[test_res, 'rate_vec']))
print("=====================================================================")
for r in top_N_recomm:
    print('Restaurant id: {}'.format(pd_business.loc[r].name))
    print('Food categories: {}'.format(pd_business.loc[r, 'Restaurant_categories']))
    print('Attributes: {}'.format(pd_business.loc[r, 'attributes_list']))
    print('Location: {}'.format(pd_business.loc[r, 'location']))
    print('Rate, # of rates: {}'.format(pd_business.loc[r, 'rate_vec']))
    print("=====================================================================")

Restaurant id: VA95qAV2PRH9gtTi2Vfv4Q
Food categories: ['food', 'salad', 'pizza', 'italian', 'coffee & tea']
Attributes: ['RestaurantsReservations', 'WiFino', 'NoiseLevelaverage', 'OutdoorSeating', 'RestaurantsAttirecasual', 'Alcoholfull_bar', 'RestaurantsDelivery', 'RestaurantsTakeOut', 'BikeParking', 'RestaurantsGoodForGroups', 'GoodForMeallunch', 'GoodForMealdinner', 'Caters', 'BusinessParkingstreet', 'RestaurantsPriceRange22']
Location: ['Toronto', 'ON']
Rate, # of rates: [3.5, 289]
Restaurant id: MUjlAtBNJR--7ai5pITbMg
Food categories: ['food', 'italian', 'coffee & tea']
Attributes: ['OutdoorSeating', 'Alcoholfull_bar', 'Ambiencetrendy', 'RestaurantsGoodForGroups', 'RestaurantsAttirecasual', 'NoiseLevelaverage', 'BikeParking', 'RestaurantsTakeOut', 'Caters', 'BusinessParkinggarage', 'BusinessParkingstreet', 'RestaurantsPriceRange23', 'RestaurantsReservations', 'WiFino', 'GoodForMealdinner']
Location: ['Toronto', 'ON']
Rate, # of rates: [3.0, 39]
Restaurant id: 6LR0Eg5EAMEIES7CY7IT

In [26]:
# print(pd_business.loc[test_res])
# print("=====================================================================")
# for r in top_N_recomm:
#     print(pd_business.loc[r])
#     print("=====================================================================")

In [27]:
# print("intersection of recommended with real data: {}".format(contentModel.simpleRecall(test_user, test_res, 1000)))

In [28]:
contentRecall = 0
for rand in randList:
    test_user = pd_review.iloc[rand, 0]
    test_res = pd_review.iloc[rand, 1]
    contentRecall += contentModel.simpleRecall(test_user, test_res, 1000)
print("recall of content base model for 10 test data: {}".format(contentRecall))

recall of content base model for 10 test data: 0.064


In [29]:
# for r in userToRes_dict[test_user]:
#     print(pd_business.loc[r])
#     print("=====================================================================")

In [30]:
# for r in userToRes_dict[test_user]:
#     print('Restaurant id: {}'.format(pd_business.loc[r].name))
#     print('Food categories: {}'.format(pd_business.loc[r, 'Restaurant_categories']))
#     print('Attributes: {}'.format(pd_business.loc[r, 'attributes_list']))
#     print('Location: {}'.format(pd_business.loc[r, 'location']))
#     print('Rate, # of rates: {}'.format(pd_business.loc[r, 'rate_vec']))
#     print("=====================================================================")

# User-Based Collaborative Recommendation Model

In [31]:
import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error

In [32]:
allUserList = list(set(pd_review['user_id']))
allResList = list(set(pd_review['business_id']))
print(len(allUserList))
print(len(allResList))

221182
43756


In [33]:
# from scipy.sparse import csc_matrix
# M = csc_matrix((len(allUserList), len(allResList)), dtype=np.int8)

In [34]:
dict1 = {'car': 0.1, 'dog':0.3, 'tiger':0.5, 'lion': 0.1, 'fish':0.2}
dict2 = {'goat':0.3, 'fish':0.3, 'shark':0.4, 'dog':0.3}
keys = list(dict1.keys() | dict2.keys())
print(keys)
np.corrcoef(
    [dict1.get(x, 0) for x in keys],
    [dict2.get(x, 0) for x in keys])[0, 1]

['shark', 'lion', 'goat', 'dog', 'fish', 'car', 'tiger']


-0.3806394141919143

In [35]:
dict1 = {'LkMtMHVetws5_7QfRjPtlg': 4.0, 'ii8sAGBexBOJoYRFafF9XQ': 5.0, 'ZkGDCVKSdf8m76cnnalL-A': 5.0}
dict2 = {'LkMtMHVetws5_7QfRjPtlg': 4.0, '6Wvn57wjSrVX7gJM37CIZw': 4.0, 'BjH8Xepc10i6OhCDQdX6og': 3.0}
keys = list(dict1.keys() | dict2.keys())
print(keys)
np.corrcoef(
    [dict1.get(x, 0) for x in keys],
    [dict2.get(x, 0) for x in keys])[0, 1]

['ii8sAGBexBOJoYRFafF9XQ', 'BjH8Xepc10i6OhCDQdX6og', 'ZkGDCVKSdf8m76cnnalL-A', '6Wvn57wjSrVX7gJM37CIZw', 'LkMtMHVetws5_7QfRjPtlg']


-0.6974927175058553

In [36]:
import warnings
warnings.filterwarnings("ignore")

class CollabRecomm:
    def __init__(self, pd_review, userToRes_dict, resToUser_dict):
        self.pd_review = pd_review.set_index(['user_id', 'business_id'])
        self.userToRes_dict = dict(userToRes_dict)
        self.resToUser_dict = dict(resToUser_dict)
        self.resIndex = {res: index for index, res in enumerate(resToUser_dict.keys())}
    
    def cal_Pearson(self, user1, user2):
#         num_res = len(self.resIndex)
#         v1 = np.zeros((num_res,))
#         v2 = np.zeros((num_res,))
#         for r in self.userToRes_dict[user1]:
#             v1[self.resIndex[r]] = self.pd_review.loc[(user1, r),'stars'].iloc[0]
        
#         for r in self.userToRes_dict[user2]:
#             v2[self.resIndex[r]] = self.pd_review.loc[(user2, r),'stars'].iloc[0]
            
#         PearsonSim = 1 - pairwise_distances([v1, v2], metric="correlation")[0,1]
        v1 = {r:self.pd_review.loc[(user1, r),'stars'].iloc[0] for r in self.userToRes_dict[user1]}
        v2 = {r:self.pd_review.loc[(user2, r),'stars'].iloc[0] for r in self.userToRes_dict[user2]}
        
        r_keys = list(v1.keys() | v2.keys())
        PearsonSim = np.corrcoef([v1.get(r, 0) for r in r_keys], [v2.get(r, 0) for r in r_keys])[0, 1]
        return PearsonSim
    
    def makeRecomm(self, userId, resId, top_N, top_K):
        sim_list = [(u, self.cal_Pearson(userId, u)) for u in self.resToUser_dict[resId] if u!=userId and u in self.userToRes_dict]
        sim_list.sort(key=lambda tp:tp[1], reverse=True)
        # select top_K similar users
        top_k_users = [tp[0] for tp in sim_list[:top_K]]
        print("top k similar users: {}".format(top_k_users[:5]))
        # return most intersected of top_k similar users but not yet rated by userId
        mostEaten = defaultdict(int)
        for u in top_k_users:
            for r in self.userToRes_dict[u]:
                if r != resId:
                    mostEaten[r] += 1
        
        mostEatenList = sorted(mostEaten.items(), key=lambda tp:tp[1], reverse=True)
        
        return [tp[0] for tp in mostEatenList[:top_N]]
    
    def simpleRecall(self, userId, resId, top_N, top_K):
        '''
        # restaurants that user has been that are also in top_N recommendations / top_N 
        '''
        top_N_recomm = set(self.makeRecomm(userId, resId, top_N, top_K))
        return len(top_N_recomm.intersection(set(self.userToRes_dict[userId]))) / top_N

In [58]:
test_user = pd_review.iloc[randomTest, 0]
test_res = pd_review.iloc[randomTest, 1]
collabModel = CollabRecomm(pd_review, userToRes_dict, resToUser_dict)
top_N_recomm = collabModel.makeRecomm(test_user, test_res, 5, 10)
print(top_N_recomm)

top k similar users: ['7ft5jiIKC8CWl_Ez9ye1_g', 'Kyl3MxItnaEScrEJccAOww', '3PvMBYkQ6RxvZpsTnM6YLg', 'EfMjxxxxLKP4HIwvnU7r-Q', 'TUDW2iLLKYih1Y8PzRHMrg']
['g1j8COaztNostGlxssCCVA', '4MQaqhBUJCpghGukSWNq7w', 'WqXrmUVy5Ay3Scn4GPM9Ug', 's11Ym0QX2Am2uwtnyou4GA', '9fMlGyhfhrob_O1DER3YWw']


In [59]:
for r in top_N_recomm:
    print('Restaurant id: {}'.format(pd_business.loc[r].name))
    print('Food categories: {}'.format(pd_business.loc[r, 'Restaurant_categories']))
    print('Attributes: {}'.format(pd_business.loc[r, 'attributes_list']))
    print('Location: {}'.format(pd_business.loc[r, 'location']))
    print('Rate, # of rates: {}'.format(pd_business.loc[r, 'rate_vec']))
    print("=====================================================================")

Restaurant id: g1j8COaztNostGlxssCCVA
Food categories: ['salad', 'italian', 'sandwiches', 'food']
Attributes: ['RestaurantsPriceRange21', 'RestaurantsAttirecasual', 'Caters', 'BusinessParkingstreet', 'BikeParking', 'RestaurantsTakeOut', 'NoiseLevelaverage', 'GoodForKids', 'OutdoorSeating', 'RestaurantsGoodForGroups', "Alcohol'none'", 'WiFino']
Location: ['Toronto', 'ON']
Rate, # of rates: [4.0, 8]
Restaurant id: 4MQaqhBUJCpghGukSWNq7w
Food categories: ['american (traditional)', 'gastropubs', 'canadian (new)', 'sports bars', 'arts & entertainment', 'bars', 'nightlife']
Attributes: ['RestaurantsTakeOut', 'RestaurantsAttirecasual', 'Alcoholfull_bar', 'Ambiencecasual', 'RestaurantsGoodForGroups', 'HasTV', 'RestaurantsReservations', 'BikeParking', 'WiFifree', 'Musicbackground_music', 'NoiseLevelloud', 'RestaurantsPriceRange22', 'HappyHour', 'GoodForMeallatenight', 'GoodForMeallunch', 'GoodForMealdinner', 'BestNightsfriday', 'BestNightsthursday', 'BestNightssaturday', 'BusinessParkingstreet'

In [60]:
# print(pd_business.loc[test_res])
# print("=====================================================================")
# for r in top_N_recomm:
#     print(pd_business.loc[r])
#     print("=====================================================================")

In [61]:
# print("intersection of recommended with real data: {}".format(collabModel.simpleRecall(test_user, test_res, 1000, 2000)))

In [62]:
collabRecall = 0
for rand in randList:
    test_user = pd_review.iloc[rand, 0]
    test_res = pd_review.iloc[rand, 1]
    collabRecall += collabModel.simpleRecall(test_user, test_res, 1000, 2000)
print("recall of collaborative base model for 10 test data: {}".format(collabRecall))

top k similar users: ['tZ_oKXjo8Y6DXgRPExO2iA', 'QyScH8Ef-7RosWQch4Ikcg', '-4VsgXM9Rg8Hh91VDG68hA', 'POj-rnht2EtX9VOQwNiNnQ', 'RQLoA7Z_M5T8iYnsFHwzYg']
top k similar users: ['om74dhhj7cL0uVGB6w9t3w', 'M2BP67TtjjpbuQ2-X3WKFQ', 'eslg8gWc8aC01H2utWw4_Q', 'nnagN-m8kL2O-jDCnypexg', 'yYTl4SYmu53ZujvtikGgRw']
top k similar users: ['x4pXPFk0lv02iM0rLMBb6A', '5wKlVEkZMYmPy6e9p9kPpA', 'J2wmUEv2n5hdPd5-OUw61A', 'ZcykwbeEMTtxfUp6PPiiLg', 'LDu8Kn3k1CZHY5sEq3bLPQ']
top k similar users: ['ueOKZidbQK2bYl7ntP6ugQ', 'ozTc_b2kVDf1yYvA74YDEA', 'GbzADFMTS8IZOCjhBm_9CA', 'fjNNUJS0dtMr7W0hHMiNVw', 'jfGVUAQ7RL4EFdAio8iHOQ']
top k similar users: ['OmT5GuNNYDG1nycaf4-exQ', 'SsEnMoB4bsN95ZRXxCh5-Q', 'FTFlzHEQ8jeEf4az0zkPFQ', 'jJ8g0lv0Jv4G3EEJgpQU9w', 'avQbuXb3ObMmSwGDcNBMjQ']
top k similar users: ['PEwOXAd4Wj4WbEoKHpthDQ', '8jPhhu9WK1E8Da408S4Hig', 'UPk3wWPYbn-hyeGOE0IpAQ', 'lUCWP1s2jW10_ELo3PNbhg', 'zeeYly1Qe5eN4NraMQqvwQ']
top k similar users: ['U98mZaQAsHXym6FyhGCZdA', 'Q5pO4T7zv2FN1wzKQEyRqQ', 'ioav4-vdhUOp1